# Optimal traffic scheduling in distributed networks

In [2]:
# standard packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Custom packages:
from optimal_traffic_scheduler import optimal_traffic_scheduler
from distributed_network import distributed_network, client_node
from ots_visu import ots_gt_plot

In [3]:
# Same configuration for all nodes:
setup_dict = {}
setup_dict['v_max'] = 20  # mb/s
setup_dict['s_max'] = 30  # mb
setup_dict['dt'] = 1  # s
setup_dict['N_steps'] = 20
setup_dict['v_delta_penalty'] = 0

ots_1 = optimal_traffic_scheduler(setup_dict, name='ots_1')
ots_2 = optimal_traffic_scheduler(setup_dict, name='ots_2')
ots_3 = optimal_traffic_scheduler(setup_dict, name='ots_3')
ots_4 = optimal_traffic_scheduler(setup_dict, name='ots_4')
ots_5 = optimal_traffic_scheduler(setup_dict, name='ots_5')

In [5]:
source_fun = []

data = np.convolve((6*np.random.rand(500, 3)).ravel(), np.ones(5)/5, mode='same').reshape(500, 3)
a = 1*np.array([1, 3, 5])
data = a*np.ones((500, 3))


def inputs(k, i):
    return [data[[k+m], i].reshape(1, 1) for m in range(setup_dict['N_steps'])]


source_fun = [lambda k: inputs(k, 0), lambda k: inputs(k, 1), lambda k: inputs(k, 2)]
def target_fun(k): return (setup_dict['N_steps']*[np.array([[0]])], setup_dict['N_steps']*[np.array([[0]])])

In [6]:
input_node_1 = client_node(setup_dict['N_steps'], name='input_node_1', source_fun=source_fun[0])
input_node_2 = client_node(setup_dict['N_steps'], name='input_node_2', source_fun=source_fun[1])
input_node_3 = client_node(setup_dict['N_steps'], name='input_node_3', source_fun=source_fun[2])
input_node_4 = client_node(setup_dict['N_steps'], name='input_node_4', source_fun=source_fun[2])
output_node_1 = client_node(setup_dict['N_steps'], name='output_node_1', target_fun=target_fun)
output_node_2 = client_node(setup_dict['N_steps'], name='output_node_2', target_fun=target_fun)
output_node_3 = client_node(setup_dict['N_steps'], name='output_node_3', target_fun=target_fun)
output_node_4 = client_node(setup_dict['N_steps'], name='output_node_4', target_fun=target_fun)

In [7]:
circuits = [
    {'route': [input_node_1, ots_1, ots_2, ots_3, ots_4, output_node_1]},
    {'route': [input_node_2, ots_3, ots_5, ots_4, ots_1, output_node_2]},
    {'route': [input_node_3, ots_1, ots_2, ots_3, ots_4, output_node_3]},
]

In [12]:
dn = distributed_network(circuits)

In [13]:
dn.nodes['con_in'] = dn.nodes['con_in']*1
dn.nodes['con_in'] = dn.nodes['con_in']*1
dn.nodes

,node,name,con_in,n_in,con_out,n_out_buffer,n_out_circuit,io_mapping,output_partition
0,<distributed_network.client_node object at 0x7...,input_node_1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[True, False, False, False, False, False, Fals...",None,1,None,None
1,<optimal_traffic_scheduler.optimal_traffic_sch...,ots_1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]",3,"[False, True, False, False, False, False, Fals...",2,3,"[0, 1, 2]","[[True, False, True], [False, True, False]]"
2,<optimal_traffic_scheduler.optimal_traffic_sch...,ots_2,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",2,"[False, False, True, False, False, False, Fals...",1,2,"[0, 1]","[[True, True]]"
3,<optimal_traffic_scheduler.optimal_traffic_sch...,ots_3,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]",3,"[False, False, False, True, False, False, True...",2,3,"[0, 1, 2]","[[True, False, True], [False, True, False]]"
4,<optimal_traffic_scheduler.optimal_traffic_sch...,ots_4,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]",3,"[False, False, False, False, True, False, Fals...",3,3,"[0, 1, 2]","[[True, False, False], [False, True, False], [..."
5,<distributed_network.client_node object at 0x7...,output_node_1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,"[False, False, False, False, False, False, Fal...",None,0,None,None
6,<distributed_network.client_node object at 0x7...,input_node_2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[False, False, False, False, False, True, Fals...",None,1,None,None
7,<optimal_traffic_scheduler.optimal_traffic_sch...,ots_5,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",1,"[False, False, False, False, False, False, Fal...",1,1,[0],[[True]]
8,<distributed_network.client_node object at 0x7...,output_node_2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[False, False, False, False, False, False, Fal...",None,0,None,None
9,<distributed_network.client_node object at 0x7...,input_node_3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[False, False, False, False, False, False, Fal...",None,1,None,None
